In [1]:
import subprocess
import glob
import pandas as pd
import numpy as np

In [13]:
files = glob.glob("/storage/home/nsl5160/scratch/brook_trout/trimmed/*_val_1.fq.gz")

Excluding the blank in the assemblies:

In [21]:
files = [i for i in files if "BLANK" not in i]

In [28]:
for file in files:
    task = "mitoz"
    mem = "128"
    cpus = "20"
    inprefix = file.rsplit("_", 2)[0]
    r1 = inprefix + "_val_1.fq.gz"
    r2 = inprefix + "_val_2.fq.gz"
    outprefix = file.rsplit("_", 2)[0].split("/", 8)[7]
    outpath = "/storage/home/nsl5160/scratch/brook_trout/mitoassembly/" + outprefix + "/"
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         r1 + " " + r2 + " " + 
         outpath + " " + outprefix], shell=True)

Submitted batch job 11897508
Submitted batch job 11897509
Submitted batch job 11897510
Submitted batch job 11897511
Submitted batch job 11897512
Submitted batch job 11897513
Submitted batch job 11897514
Submitted batch job 11897515
Submitted batch job 11897516
Submitted batch job 11897517
Submitted batch job 11897518
Submitted batch job 11897519
Submitted batch job 11897520
Submitted batch job 11897521
Submitted batch job 11897522
Submitted batch job 11897523
Submitted batch job 11897524
Submitted batch job 11897525
Submitted batch job 11897526
Submitted batch job 11897527
Submitted batch job 11897530
Submitted batch job 11897532
Submitted batch job 11897534
Submitted batch job 11897536
Submitted batch job 11897538
Submitted batch job 11897540
Submitted batch job 11897541
Submitted batch job 11897542
Submitted batch job 11897543
Submitted batch job 11897544
Submitted batch job 11897545
Submitted batch job 11897546
Submitted batch job 11897547
Submitted batch job 11897548
Submitted batc

In [4]:
%%bash
cd ~/scratch/brook_trout/mitoassembly/
rm circular.txt
for i in $( ls /storage/home/nsl5160/scratch/brook_trout/mitoassembly/ | grep -v "Undetermined" | grep -v "BLANK" ); do
    CIRCULAR=$( head -2 ./${i}/${i}.result/${i}.${i}.megahit.mitogenome.fa.result/summary.txt | grep "yes" | wc -l )
    if [ $( echo ${CIRCULAR} ) -ge 1 ]; then
        echo -e "${i}" \
            >> circular.txt
    fi
done

rm: cannot remove 'circular.txt': No such file or directory
head: cannot open './Tu_6/Tu_6.result/Tu_6.Tu_6.megahit.mitogenome.fa.result/summary.txt' for reading: No such file or directory
head: cannot open './VT2_E/VT2_E.result/VT2_E.VT2_E.megahit.mitogenome.fa.result/summary.txt' for reading: No such file or directory


In [6]:
%%bash
cd ~/scratch/brook_trout/mitoassembly/
rm multimito.fa
for i in $( cat ./circular.txt ); do
    DIRECTION=$( cat ./${i}/${i}.result/${i}.${i}.megahit.mitogenome.fa.result/summary.txt | grep "s-rRNA" | head -1 | tr -s " " | cut -d" " -f5 | head -1)
    SEQ=$( tail -1 ./${i}/${i}.result/${i}.${i}.megahit.mitogenome.fa.result/${i}_${i}.megahit.mitogenome.fa_mitoscaf.fa.gbf.fasta )
    if [ "${DIRECTION}" == "+" ]; then
        echo -e ">${i}" >> multimito.fa
        echo ${SEQ} >> multimito.fa
    elif [ "${DIRECTION}" == "-" ]; then
        echo -e ">${i}" >> multimito.fa
        echo ${SEQ} | tr "ATGC" "TACG" | rev >> multimito.fa
    else
        continue
    fi
done

rm: cannot remove 'multimito.fa': No such file or directory


In [1]:
!cat ~/scratch/brook_trout/mitoassembly/multimito.fa \
    ../multimito_rotated_m0_round1.fa \
    > ../multimito_mars_input.fa

In [6]:
task = "mars"
mem = "64"
cpus = "8"
infile = "../multimito_mars_input.fa"
outfile = "../multimito_mars_circularized.fa"
subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
     cpus + " ../" + task + ".sh " + 
     infile + " " + outfile], shell=True)

Submitted batch job 20628392


CompletedProcess(args=['sbatch --mem=64g --ntasks=8 ../mars.sh ../multimito_mars_input.fa ../multimito_mars_circularized.fa'], returncode=0)

In [7]:
task = "mafft"
mem = "64"
cpus = "8"
infile = "../multimito_mars_circularized.fa"
outfile = "../multimito_mars_circularized.msa"
subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
     cpus + " ../" + task + ".sh " + 
     infile + " " + outfile], shell=True)

Submitted batch job 20633781


CompletedProcess(args=['sbatch --mem=64g --ntasks=8 ../mafft.sh ../multimito_mars_circularized.fa ../multimito_mars_circularized.msa'], returncode=0)

In [9]:
!head -500 ../multimito_mars_circularized.msa

>29012_19
gtccacatgacaaaaactagcacctttcgcacttataattcaggtagcccccgccattga
ctcttccttacttgttgcattgggccttatatcgatactcgtcggaggatgaggaggact
taaccaaacccaactacgtaaaattttagcatattcttcaattgcccacctagggtggat
agtgctgattttacaattcgcaccttccctgacactcctcagccttttcctatatattat
cataacatcttcagcattcctcgcactgaaaaccaacaactctttaactattaataccct
agcaacttcatgaactaagtccccggctctcgccccactaaccgctctagtgctactgtc
cctagggggcctaccccctctctcgggatttatacctaaatgacttattttacaagaact
tacaaagcaagaacttccactacctgccacactagctgctataatagccctccttagtct
ctacttttatttacgcctctgttacgctatagccctaactatttatcccaacaccttaac
tgctgcggccccatgacgccttgactttactatcattaccctgcccctttcaattgttac
tattctagccctagccctactccccctcactccagctgtaactgcaatgttaaccttgta
aagggcttaggatagcattaagaccaagagccttcaaagctctaagcgggagtgaaaatc
tcccagcccttgttaaaacttgcaggactttatcccacatcttctgaatgcaacccagac
actttaattaagctaaagcctttctaggtgggaaggcctcgatcctacaaattcttagtt
aacagctaagcgctctatccagcgagcatccatctactttcccccgccgtccggggggag
cgaggcgggggaaagccccggcaggctattagcctacttctttagatttgcaatctaacg
tgtaattcactaca

In [46]:
!grep "^>" ../multimito_mars_circularized.msa \
    | sed "s/>//g" > ../allsamples.txt

In [47]:
meta = pd.read_csv("../wilmurt_meta.csv")
mitoseqs = pd.read_csv("../allsamples.txt", header = None)[0].tolist()
adirondack = meta[meta["Adirondack?"] == "Y"][meta["Novogene_ID"].isin(mitoseqs)].reset_index(drop=True)

/tmp/ipykernel_1491075/260210863.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adirondack = meta[meta["Adirondack?"] == "Y"][meta["Novogene_ID"].isin(mitoseqs)].reset_index(drop=True)


In [49]:
adirondack[["Novogene_ID"]].to_csv("../adirondack.txt", header = None, index = False)

In [50]:
%%bash
source ~/.bashrc
conda activate assembly
seqkit grep -n -f ../adirondack.txt \
    ../multimito_mars_circularized.msa > ../adirondack.msa